In [74]:
using Plots
using Optim
using LinearAlgebra
using DataFrames
using GLM
using CSV
using TypedTables
include("markovvec.jl")

markov (generic function with 1 method)

In [ ]:
N = 10000
m = 1.67e-27
Tinit = 300
step = 10
Tfinal = 500
T = [Tinit:step:Tfinal;]
iter = 1000
velocities = 2000
ββ = []
ratio = []
EE = []
vv = []
ΔEE = []
for t in T
    println("-------------------------------------------------------------------------")
    println("T = ",t)
    ΔE, ratio_tentativas,energies,velocidades, β = markov(N,m,t,velocities,iter)
    push!(ΔEE,ΔE)
    push!(ββ,β)
    push!(vv,velocidades)
    push!(ratio,ratio_tentativas)
    push!(EE,energies)
end

-------------------------------------------------------------------------
T = 300
loop: 1
loop: 2
loop: 3
loop: 4
loop: 

In [ ]:
data = DataFrame("β" => ββ,"ratio" => ratio,"E" => EE,"v" => vv,"ΔE" => ΔEE)
CSV.write("dados/simulação com T = [$Tfinal:$step:$Tfinal;] iter = $iter, N = $N, velocidade = $velocities.csv",data)

In [ ]:

#data = CSV.read("dados/simulação com T = [500:5:500;] iter = 20, N = 4000.csv",DataFrame)
#data.E = Matrix{Float64}(data.E)

In [ ]:
scalar = []
for i in 1:N
    push!(scalar,norm(vv[1][i,:]))
    end
scalar

In [ ]:
Plots.xlabel!("Velocidades escalar")
Plots.histogram(scalar)

Para plotar a curva, vai exigir com que eu selecione o intervalo de valores e isso eu posso fazer usando a velocidade que dei de entrada

In [ ]:
final_energy = [mean(EE[i]) for i in 1:length(EE)]

In [ ]:
Plots.scatter(T,final_energy)
Plots.title!("T vs U")
Plots.xlabel!("temperatura (K)")
Plots.ylabel!("energia (J)")

Podemos perceber que a energia e a temperatura possuem uma relação linear. Logo, eu decidi então fazer um ajuste linear

In [ ]:
t = Table(energy = final_energy,temperature = T)
t

In [ ]:
ols = lm(@formula(energy ~ temperature),t)

In [ ]:
plot!(T,predict(ols))

In [ ]:
α = coef(ols)[2]
k_B = 1.38e-23
2*α/(3*k_B) # não tá muito certo fazer isso, mas....

In [ ]:
sum(EE[1])

In [ ]:
sum(ℯ.^(-ββ[1].*EE[1]))

In [ ]:
sum(EE[1])

In [ ]:
ℯ^(-ββ[1]*sum(EE[1]))

In [ ]:
ββ[1]

In [ ]:
logZ=-sum(EE[1]) * ββ[1]

In [ ]:
exp(-sum(EE[1]) * ββ[1])

## Calculando a função de Partição:

$$Z = \sum_i e^{-\beta E_i}$$

Para evitar underflow numérico
Está sendo usado

$$\log Z = -\sum_i \beta E_i$$

In [ ]:
logZ = []
for i in 1:length(ββ)
    push!(logZ,sum(-ββ[i].*EE[i]))
    end
logZ

In [ ]:
final_energy

Obtendo a energia média pela função de partição

$$\bar E = -\frac{\partial \ln Z}{\partial \beta}$$

In [ ]:
-diff(logZ)./diff(ββ)

In [ ]:
mean(EE[1])

## Calcular a entropia

Estou usando a entropia de Shannon que é dado por:

$$S = -k_B \sum_j P_j \ln P_j$$

onde $$P_j = \frac{\exp (-\beta E_j)}{\sum_k \exp (-\beta E_k)}$$

ou

$$P_j = \frac{\exp (-\beta E_j)}{Z}$$

Como está sendo usado $\log Z$ para evitar underflow numérico

$$\log P_j = -\beta E_j - \log Z$$

Obtendo as probabilidades

In [ ]:
logP = []
for i in 1:length(Z)
    for j in 1:length(EE[i])
        push!(logP,-ββ[i] * EE[i][j] .- Z[i])
    end
end

P = logP  # Aplica exp. elemento por elemento


Obtendo a entropia $S$.

In [ ]:
S = []
for i in 1:length(P)
    push!(S,-k_B*sum(P[i].*log.(P[i])))
end
S

A temperatura vai ser dada por

$$T = \frac{\partial \bar E}{\partial S}$$

In [ ]:
U = [0; -diff(log.(Z))./(diff(ββ))]

In [ ]:
dU = [0;diff(U)]

In [ ]:
dS = [0;diff(S)]

In [ ]:
T